In [41]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
import torch.nn.functional as F


In [42]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
GeForce RTX 2080 with Max-Q Design


In [43]:
# data_dir = '../../../../BottleStoodUp_atNight/Positive/'        #For the work laptop
data_dir = '../../../Images/BottleStoodUp_atNight/Positive'        #For the home laptop

In [44]:
# transform_characteristics = transforms.Compose([transforms.Resize(255),
#                                 transforms.CenterCrop(224),
#                                 transforms.ToTensor()])

transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])
dataset = datasets.ImageFolder(data_dir, transform=transform_characteristics)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

Building the network in which the pre-trained model will be loaded.

In [45]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()        
        # 32, 1, 224, 224.  Batch size, input channels, shape of the image.
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image divided by 2 approximately.   
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        
        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> 15, 110, 44, 44. Batch size, channel output, output image shape.
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # Dimension rseult: -> 15, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # Dimension rseult: -> 15, 450, 1, 1
        )
        
        # Initial dimension for this part of the model: 15 , 450, 1, 1
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(output_channels_layer3, output_channels_layer2, 8),  # Dimension rseult: -> 15, 225, 8, 8
            nn.ReLU(),
            nn.ConvTranspose2d(output_channels_layer2, output_channels, kernel_size, stride=stride_val, padding=padding_val, output_padding=2), # Dimension rseult: -> 15, 110, 44, 44
            nn.ReLU(),
            nn.ConvTranspose2d(output_channels, input_channels, kernel_size, stride=stride_val, padding=padding_val, output_padding=2), # Dimension rseult: -> 15, 1, 224, 224
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
 
# Note: nn.MaxPool2d -> use nn.MaxUnpool2d, or use different kernelsize, stride etc to compensate...
# Input [-1, +1] -> use nn.Tanh

In [46]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device

device = get_device()
print(device)

cuda:0


In [47]:
model = Autoencoder()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3, 
                             weight_decay=1e-5)

In [48]:
model.to(device)

Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(450, 225, kernel_size=(8, 8), stride=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(225, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (3): ReLU()
    (4): ConvTranspose2d(110, 3, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (5): Sigmoid()
  )
)

Loading the model

In [49]:
filepath = "../../../BottlesAnomalies_TFM/models/pytorchModels/PytorchModel_withCUDA"
# For loading the model 
model.load_state_dict(torch.load(filepath))
model.eval()

Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(450, 225, kernel_size=(8, 8), stride=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(225, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (3): ReLU()
    (4): ConvTranspose2d(110, 3, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (5): Sigmoid()
  )
)

Extracting the layers' weights of the model that has been loaded.

In [50]:
layers_weights_list = []
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        weights = m.weight
        layers_weights_list.append(weights)
print("The number of layers that the loaded model has, is: ", len(layers_weights_list))

The number of layers that the loaded model has, is:  3


Copying the model layers' weights from one model to another. 

In [51]:
class Autoencoder_latentSpace(nn.Module):
    def __init__(self):
        super().__init__()        
        # 32, 1, 224, 224.  Batch size, input channels, shape of the image.
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image divided by 2 approximately. 
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> 15, 110, 44, 44. Batch size, channel output, output image shape.
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # Dimension rseult: -> 15, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # Dimension rseult: -> 15, 450, 1, 1
        )

    def forward(self, x):
        # print("This is the forward function")
        encoded = self.encoder(x)
        return encoded
    
    def show_modules(self):
        print("This is the show modules function")
        i = 0
        for m in self.modules():
            print(m)
            print("i is: ", i)
            print("print the next module")
            i = i +1
            
    def show_one_layer_weights(self, index):
        print("This is the one layer show function")
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if i == index:
                    print("i is: ", i)
                    print("The weights are: ", m.weight)
                i = i +1
    
    def update_weights(self):
        print("updating weights function")
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight = layers_weights_list[i]
                i = i +1

                                        
 

In [52]:
model_encoder = Autoencoder_latentSpace()
model_encoder.to(device)
# model.eval()

Autoencoder_latentSpace(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
)

In [53]:
model_encoder.update_weights()

updating weights function


In [54]:
model_encoder.show_one_layer_weights(0)

This is the one layer show function
i is:  0
The weights are:  Parameter containing:
tensor([[[[-1.2588e-14, -2.4193e-14, -2.0195e-14,  ..., -1.7424e-14,
           -1.8165e-14, -2.0065e-14],
          [-4.0820e-14, -6.4369e-14, -5.6829e-14,  ..., -5.9112e-14,
           -5.9453e-14, -4.3775e-14],
          [-3.8931e-14, -5.8424e-14, -6.1809e-14,  ..., -5.6119e-14,
           -5.4058e-14, -5.3529e-14],
          ...,
          [-4.3402e-14, -6.5496e-14, -6.1202e-14,  ..., -5.4887e-14,
           -5.3833e-14, -4.7419e-14],
          [-4.1414e-14, -6.1010e-14, -6.2996e-14,  ..., -5.9068e-14,
           -5.5303e-14, -5.9356e-14],
          [-3.1195e-14, -5.2059e-14, -5.6881e-14,  ..., -4.7215e-14,
           -5.4061e-14, -4.4563e-14]],

         [[-1.1739e-14, -1.9304e-14, -1.9236e-14,  ..., -1.7454e-14,
           -1.9047e-14, -1.7575e-14],
          [-4.0282e-14, -6.3855e-14, -6.0149e-14,  ..., -5.6298e-14,
           -5.3430e-14, -6.7751e-14],
          [-3.9354e-14, -6.1378e-14, -6.11

At this point, the created "model_encoder" model contains the weights of the encoder part of the pre-trained Autoencoder model. 

## Obtaining the KDE representation of the training set images

In [55]:
########################################################
# Calculate KDE using sklearn
from sklearn.neighbors import KernelDensity

#Get encoded output of input images = Latent space
# encoded_images = model_encoder(images)
encoded_images = []

for i in range(len(dataset)):
    X = dataset[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda()            # Because GPU is being used
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    encoded_images.append(Y)


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [68]:
print(Y)

tensor([[[ 1.9623e+00]],

        [[-1.3005e+00]],

        [[ 1.3377e+01]],

        [[ 3.4286e-01]],

        [[ 3.9430e-01]],

        [[-3.1119e+00]],

        [[ 4.1906e+00]],

        [[ 4.5664e-01]],

        [[-5.7354e-02]],

        [[ 1.2062e+00]],

        [[ 2.3741e+00]],

        [[ 2.3798e+00]],

        [[ 1.6479e+00]],

        [[ 1.5264e+00]],

        [[-1.9825e-01]],

        [[ 2.6015e+00]],

        [[ 8.1773e-01]],

        [[-1.6988e+01]],

        [[ 1.3998e+02]],

        [[ 1.7023e+00]],

        [[ 4.1302e-01]],

        [[ 3.8113e-01]],

        [[ 2.7575e+00]],

        [[ 1.2269e+00]],

        [[-1.5262e-02]],

        [[ 1.0461e+00]],

        [[-6.0094e+01]],

        [[ 2.0180e+00]],

        [[-1.6021e+00]],

        [[ 7.7901e-01]],

        [[ 3.0199e-01]],

        [[ 1.0352e+00]],

        [[-1.7000e-02]],

        [[ 3.7251e-01]],

        [[ 6.0550e-01]],

        [[ 1.9593e+00]],

        [[ 2.8144e+00]],

        [[ 9.0850e-01]],

        [[ 3

Convert "encoded_images" to a np array

In [56]:
np_encoded_images = []
for i in range (len(encoded_images)):
    # np_conversion = encoded_images[i].detach().numpy()        # If not using GPU
    np_conversion = encoded_images[i].cpu().detach().numpy()    # If using GPU
    np_encoded_images.append(np_conversion)
np_encoded_images = np.array(np_encoded_images)
print(type(np_encoded_images))

<class 'numpy.ndarray'>


In [57]:
print(np_encoded_images.shape)

(179, 450, 1, 1)


Also, see above the shape of the representation of the original images has been lowered to (1, 1) as specified in the model structure. The number 450, on the other hand, corresponds to the channels of the image; this value started at 3 and layer by layer it incremented until reaching 450.

Now, we have to flatten the data in order to apply kernel density on it.

In [58]:
model_encoder_output_shape = (450,1,1)
print(model_encoder_output_shape)

(450, 1, 1)


In [59]:
out_vector_shape = model_encoder_output_shape[0]*model_encoder_output_shape[1]*model_encoder_output_shape[2]
encoded_images_vector = [np.reshape(img, (out_vector_shape)) for img in np_encoded_images]


In [60]:
kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(encoded_images_vector)

The above function fits a kernel density estimation to the data that is provided, that is, to the "encoded_images_vector" variable. It does so using a Guassian kernel of bandwidth 0.2.

The badnwidth parameter affects on how the selected kernel will fit each sample of the given data. For example for the case in which the kernel is a Gaussian distribution, the bandwidth parameter would affect in how thin or wide is the Gaussian distribution.

At this point in the variable "kde" we have some numbers that are the result of fitting Gaussian functions to the given data points in the variable "encoded_images_vecotr". We will use the "kde" variable later for scoring with it, some given data points; the scoring will be given depending on how similar are the given data points to the ones that it had estimated.

Here below, it is shown the kde values corresponding to each encoded sample:

In [61]:
density_vals = kde.score_samples(encoded_images_vector)
print(density_vals)

[305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 306.23048203 306.23048203 306.23048203 306.23048203 306.23048203
 306.23048203 306.23048203 306.23048204 306.23048204 306.23048203
 306.23048203 306.23048203 306.23048203 306.23048203 306.23048203
 306.23048

Notice that the above density values are pretty much the same among them. This has to do with the new model that was trained. The previous model from the "Pt_latentSpace_DS1" program, makes these value to be more different among them.

From here, the mean and standard deviation of these values are computed

In [62]:
average_density = np.mean(density_vals)
stdev_density = np.std(density_vals)
print("The avg of the density values is: ", average_density)
print("The stdev_density of the density values is: ", stdev_density)

The avg of the density values is:  305.69224174186684
The stdev_density of the density values is:  0.28873460688090674


Now, it will be shown the density mean and std deviation of the set of anomalies samples

In [63]:
data_anomalies = '../../../Images/BottleStoodUp_atNight/Anomalies2.0'      #This is for the home laptop
# data_anomalies = '../../../Images/BottleStoodUp_atNight/Anomalies2.0'      #This is for the work laptop
transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])
dataset_anomalies = datasets.ImageFolder(data_anomalies, transform=transform_characteristics)
dataloader_anomalies = torch.utils.data.DataLoader(dataset_anomalies, batch_size=32, shuffle=True)

In [64]:
#Get encoded output of input images = Latent space
encoded_anomalies_images = []

for i in range(len(dataset_anomalies)):
    X = dataset_anomalies[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda()            # Because GPU is being used
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    encoded_anomalies_images.append(Y)


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [65]:
np_encoded_anomaly_images = []
for i in range (len(encoded_anomalies_images)):
    # np_conversion = encoded_anomalies_images[i].detach().numpy()      # If GPU is not used
    np_conversion = encoded_anomalies_images[i].cpu().detach().numpy()    # If GPU is used
    np_encoded_anomaly_images.append(np_conversion)
np_encoded_anomaly_images = np.array(np_encoded_anomaly_images)
print(type(np_encoded_anomaly_images))

<class 'numpy.ndarray'>


In [66]:
encoded_anomaly_images_vector = [np.reshape(img_encoded, (out_vector_shape)) for img_encoded in np_encoded_anomaly_images]


In [67]:
print(encoded_anomaly_images_vector)

[array([-2.75589749e-02,  1.07601023e+00,  1.02152243e+01, -1.65061861e-01,
       -3.62640202e-01,  3.13112116e+00, -1.84429467e+00, -3.69889855e-01,
       -1.89832360e-01, -6.33630991e-01, -1.24589169e+00, -1.32160032e+00,
        8.09951901e-01, -1.76260829e+00,  2.46129394e-01, -1.12595928e+00,
        1.07498214e-01, -1.38279533e+01,  1.26676445e+02, -1.45330608e-01,
       -1.17230225e+00,  1.59639865e-02, -1.02418494e+00,  9.43357110e-01,
       -4.16581631e-02,  7.76970327e-01, -5.15992699e+01, -1.12818100e-01,
       -3.70598078e-01, -3.36706072e-01,  1.46814203e-02,  1.17073143e+00,
       -2.08103925e-01, -6.99302435e-01, -2.92120039e-01, -9.21550095e-01,
       -1.69707561e+00,  9.63824987e-01, -1.53831124e+00,  1.70668519e+00,
       -3.41923505e-01, -2.61998606e+00,  1.04283541e-03,  2.68086672e+00,
       -1.44867873e+00, -3.66729021e+00,  2.29198456e+00,  4.20181364e-01,
        8.91115785e-01,  5.86457014e-01, -9.60742235e-02, -1.07412612e+00,
        1.43417090e-01, 

In [27]:
density_vals_anomalies = kde.score_samples(encoded_anomaly_images_vector)
print(density_vals_anomalies)

[ 296.96890859   55.65270378 -595.06646246  182.74331972   75.25784751
  125.69421808]


In [28]:
average_density_anomalies = np.mean(density_vals_anomalies)
stdev_density_anomalies = np.std(density_vals_anomalies)
print("The avg of the density values is: ", average_density_anomalies)
print("The stdev_density of the density values is: ", stdev_density_anomalies)

The avg of the density values is:  23.541755870476862
The stdev_density of the density values is:  287.78593497400766


See that the std deviation along with the mean of these density values will overlap the mean of the non-anomaly images. 

Considering the situation of the overlapping commented above, given a density value, it will be built a function that:
- Assigns a percentage value according to the proximity to the mean of the non-anomaly images. For example: 
    - If the densitiy value is 305.69 (the mean of the density values of the non-anomaly images), then this density value should have 100% chance to be considered as non-anomaly.
    - If the densitiy value is 305.4014 (the mean of the density values of the non-anomaly minus the std deviation of the same set), then this density value should have 50% chance to be considered as non-anomaly.
- Assigns a percentage value according to the proximity to the mean of the anomaly images. For example: 
    - If the densitiy value is 231.6065 (the mean of the density values of the anomaly images), then this density value should have 100% chance to be considered as an anomaly image.
    - If the densitiy value is 311.0713 (the mean of the density values of the anomaly plus the std deviation of the same set), then this density value should have 50% chance to be considered as an anomaly.
- The two percentage values from above will be summed up assigning the following weights to the equation:

        = perc_NOanomaly*0.75 + perc_anomaly*0.25
        
    More weight is assigned to the non-anomaly images because there are more samples of this kind of images.
- After the weighted sum, the result will be subtracted from 100, to finally output the probability of an image to be an anomaly image. 

In [29]:
def ranges_mapper(value, leftMin, leftMax, rightMin, rightMax):
    # if(value>leftMax):
    #     return rightMax
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

In [30]:
def map_kde2prob_list(input_list):
    threshold_NOanomaly = average_density          # The mean of the density values corresponding to the non-anomaly images
    std_dev_NOanomaly = stdev_density            # The std deviation of the density values corresponding to the non-anomaly images

    threshold_anomaly = average_density_anomalies          # The mean of the density values corresponding to the anomaly images
    std_dev_anomaly = stdev_density_anomalies            # The std deviation of the density values corresponding to the anomaly images
    prob_score_list = []

    for i in range (len(input_list)):
        score_NOanomaly = input_list[i] - threshold_NOanomaly

        perc_NOanomaly = ranges_mapper(abs(score_NOanomaly), 0, std_dev_NOanomaly, 100, 50)
        if perc_NOanomaly<0:
            perc_NOanomaly = 0
        if perc_NOanomaly>100:
            perc_NOanomaly = 100
        

        score_anomaly = input_list[i] - threshold_anomaly

        perc_anomaly = ranges_mapper(abs(score_anomaly), 0, std_dev_anomaly, 100, 50)
        if perc_anomaly<0:
            perc_anomaly = 0
        if perc_anomaly>100:
            perc_anomaly = 100
        
        prob_score = 100 - (perc_NOanomaly*0.75+perc_anomaly*0.25)
        prob_score_list.append(prob_score)
    return prob_score_list

In [31]:
def map_kde2prob(value):
    threshold_NOanomaly = average_density          # The mean of the density values corresponding to the non-anomaly images
    std_dev_NOanomaly = stdev_density            # The std deviation of the density values corresponding to the non-anomaly images

    threshold_anomaly = average_density_anomalies          # The mean of the density values corresponding to the anomaly images
    std_dev_anomaly = stdev_density_anomalies            # The std deviation of the density values corresponding to the anomaly images          # The std deviation of the density values corresponding to the anomaly images

    score_NOanomaly = value - threshold_NOanomaly

    perc_NOanomaly = ranges_mapper(abs(score_NOanomaly), 0, std_dev_NOanomaly, 100, 50)
    if perc_NOanomaly<0:
        perc_NOanomaly = 0
    if perc_NOanomaly>100:
        perc_NOanomaly = 100
    # print(perc_NOanomaly)

    score_anomaly = value - threshold_anomaly

    perc_anomaly = ranges_mapper(abs(score_anomaly), 0, std_dev_anomaly, 100, 50)
    if perc_anomaly<0:
        perc_anomaly = 0
    if perc_anomaly>100:
        perc_anomaly = 100
    # print(perc_anomaly)
    prob_score = 100 - (perc_NOanomaly*0.75+perc_anomaly*0.25)
  
    return prob_score

In [32]:
def computePred(kde_value):
    pred = 0
    prob_anomaly = map_kde2prob(kde_value)/100
    if prob_anomaly > 0.5:
        pred = 1
    return pred

In [33]:
print("The prob of the kde value of being anomaly image is: ", map_kde2prob(average_density+stdev_density))
print("Given the probability, it is actually predicted as:", computePred(average_density+stdev_density))

The prob of the kde value of being anomaly image is:  49.7677651189991
Given the probability, it is actually predicted as: 0


Testing the kde thresholds 

In [34]:
test_dir = '../../../Images/BottleStoodUp_atNight/Evaluation'      #This is for the home laptop
transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])

dataset_test = datasets.ImageFolder(test_dir, transform=transform_characteristics)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=32, shuffle=True)
classes = ('non-anomaly','anomaly')

In [35]:
#Get encoded output of input images = Latent space
encoded_test_imgs = []


for i in range(len(dataset_test)):
    X = dataset_test[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda()     
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    # np_conversion = Y.detach().numpy()
    np_conversion = Y.cpu().detach().numpy()
    encoded_test_imgs.append(np_conversion)
np_encoded_test_images = np.array(encoded_test_imgs)
print(type(np_encoded_test_images))

c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


<class 'numpy.ndarray'>


In [36]:
encoded_test_images_vector = [np.reshape(img, (out_vector_shape)) for img in np_encoded_test_images]
print(len(encoded_test_images_vector))

30


In [37]:
density_vals_test = kde.score_samples(encoded_test_images_vector)
print(density_vals_test)

[-4.95558450e+02 -1.28904187e+03 -3.35437614e+03 -3.32539702e+03
 -4.22695094e+03 -5.01287596e+03 -8.01057807e+03 -3.84557941e+03
 -1.65088935e+03 -3.14852874e+03  2.02391580e+01  3.29612692e+01
  2.20480479e+02 -3.12373699e+02 -8.87396145e+02 -1.03108680e+02
 -3.84304420e+04 -2.51749392e+04 -2.99073537e+03 -5.78096574e+02
 -6.20268781e+02 -7.79260980e+02 -9.44952368e+02 -8.83991822e+02
 -1.12114144e+03 -3.32372090e+03 -1.87427539e+03 -3.12592947e+03
 -3.57855271e+03 -2.25271327e+03]


The above shown values are REALLY strange. It was not expected to have negative values in the density numbers!

In [38]:
prob_test = map_kde2prob_list(density_vals_test)

In [39]:
print(prob_test)

[97.54714974229441, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 75.14344854362331, 75.40913714622384, 83.55404567866452, 89.59050869301907, 100.0, 80.50106954919976, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [40]:
y_pred = []
y_true = []

# Grabbing only the first image of the anomalies dataset
X = dataset_test[0]
image_in_tensor = X[0]
 
n_features = len(image_in_tensor[0])  # Get the size of one image of the anomaly images dataset. This is supposed to be 224
for i in range(len(dataset_test)):
    X = dataset_test[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda() 
    ground_truth = X[1]

    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    np_converted_encoded_img = Y.cpu().detach().numpy()
    flattened = np.reshape(np_converted_encoded_img, (out_vector_shape))
    density = kde.score_samples([flattened])[0]
    prediction = computePred(density)
    y_pred.append(prediction) # Save Prediction
    y_true.append(ground_truth) # Save Truth
print(y_true)
print(y_pred)

c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to b

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to b

Above, it can be seen that all the images are predicted to be anomaly images. This is good for the TPR but it is terrible for the FPR metric.